<a href="https://colab.research.google.com/github/KarnB013/scamzeroed/blob/main/ScamKeywordDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
df = pd.read_table("SMSSpamCollection" , sep='\t' ,header =None)
df = df.reindex(np.random.permutation(df.index))
df.reset_index(inplace=True, drop=True)
df

,0,1
0,ham,"Also sir, i sent you an email about how to log..."
1,ham,how are you? I miss you!
2,ham,Thanx a lot 4 ur help!
3,ham,Just hopeing that wasn‘t too pissed up to reme...
4,ham,What Today-sunday..sunday is holiday..so no wo...
...,...,...
5567,ham,I'm really sorry i won't b able 2 do this frid...
5568,spam,Sunshine Hols. To claim ur med holiday send a ...
5569,ham,I am late. I will be there at
5570,spam,URGENT! Your Mobile number has been awarded wi...


In [5]:
df[0]=df[0].replace('ham', 0)
df[0]=df[0].replace('spam', 1)

df

,0,1
0,0,"Also sir, i sent you an email about how to log..."
1,0,how are you? I miss you!
2,0,Thanx a lot 4 ur help!
3,0,Just hopeing that wasn‘t too pissed up to reme...
4,0,What Today-sunday..sunday is holiday..so no wo...
...,...,...
5567,0,I'm really sorry i won't b able 2 do this frid...
5568,1,Sunshine Hols. To claim ur med holiday send a ...
5569,0,I am late. I will be there at
5570,1,URGENT! Your Mobile number has been awarded wi...


In [32]:
X_train, X_test, y_train, y_test = train_test_split(df[1], df[0], test_size=0.2, random_state=42)

count_vector = CountVectorizer(ngram_range=(1, 1), lowercase = True , stop_words =  'english')
X_train_count = count_vector.fit_transform(X_train)
X_test_count = count_vector.transform(X_test)
X_train_feature_list = count_vector.get_feature_names_out()
X_train_feature_list

array(['00', '000', '000pes', ..., 'zouk', 'èn', 'ú1'], dtype=object)

In [33]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
y_train_reshape = y_train.array.reshape(-1, 1)
naive_bayes.fit(X_train_count , y_train_reshape)
predictions = naive_bayes.predict(X_test_count)
predictions

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 0, ..., 1, 0, 0])

In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('Accuracy score: ', format(accuracy_score(predictions, y_test)))
print('Precision score: ', format(precision_score(predictions, y_test)))
print('Recall score: ', format(recall_score(predictions, y_test)))
print('F1 score: ', format(f1_score(predictions, y_test)))

Accuracy score:  0.9874439461883409
Precision score:  0.9448275862068966
Recall score:  0.958041958041958
F1 score:  0.9513888888888888


In [26]:
naive_bayes = SVC()
y_train = y_train.array.reshape(-1, 1)
naive_bayes.fit(X_train_count , y_train_reshape)
predictions = naive_bayes.predict(X_test_count)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# count_vector = CountVectorizer(ngram_range=(1, 1), lowercase = True , stop_words = 'english')
a = ["This is a test so do not stress out", "WIN new money free"]
# b = count_vector.fit_transform(a[0])
b = count_vector.transform(a)

c = count_vector.get_feature_names_out()
c

array(['00', '000', '000pes', ..., 'èn', 'ú1', '〨ud'], dtype=object)

In [48]:
def detect_scam_words(text):
  predictors = count_vector.transform(text)
  result = naive_bayes.predict(predictors)
  print(result)
  average = sum(result)/len(result)
  print(average)
  if average >= 0.5:
    return True
  else:
    return False

In [49]:
print(detect_scam_words(["Congratulations", "hi", "you", "prize"]))

[1 0 0 1]
0.5
True


In [41]:
import pickle

# save
with open('model.pkl','wb') as f:
    pickle.dump(naive_bayes,f)

# load
with open('model.pkl', 'rb') as f:
    naive_bayes2 = pickle.load(f)